In [198]:
# STEP 0: Import libraries and load raw data
import pandas as pd
import numpy as np
import re
url = "https://raw.githubusercontent.com/a3s5jj/Data_Projects/main/dirty_marketing_data.csv"
df = pd.read_csv(url)


df = pd.read_csv(url, dtype=str)
print("Initial rows:", len(df))
df.head()


Initial rows: 670


,Campaign_ID,Campaign Id,Date,Platform,Country,Ad_Format,Impressions,Clicks,Spend_USD,SpendUSD,Conversions,CTR,Revenue_USD,Notes,utm_source
0,CAMP016,NaN,6/16/2024,Instagram,CA,Image,3831,99,50.07,NaN,4,4.82%,NaN,top performer,cpc
1,CAMP027,Camp027,1/11/2024,Instagram,US,Image,8521,209,NaN,7636.43,12,1.95%,8521.23,NaN,email
2,NaN,NaN,21-09-2024,LinkedIn,DE,Video,5795,224,908.64,NaN,19,3.08%,6068.56,top performer,cpc
3,CAMP028,NaN,7/11/2024,Google ads,FR,Carousel,"7,590",7,6.59,NaN,1,NaN,49.99,NaN,cpc
4,NaN,NaN,12/1/2024,Google Ads,DE,Carousel,21553,-299,0,NaN,0,3.95%,NaN,needs review,email


In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 670 entries, 0 to 669
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Campaign_ID  446 non-null    object
 1   Campaign Id  135 non-null    object
 2   Date         670 non-null    object
 3   Platform     670 non-null    object
 4   Country      670 non-null    object
 5   Ad_Format    670 non-null    object
 6   Impressions  670 non-null    object
 7   Clicks       641 non-null    object
 8   Spend_USD    473 non-null    object
 9   SpendUSD     197 non-null    object
 10  Conversions  597 non-null    object
 11  CTR          550 non-null    object
 12  Revenue_USD  204 non-null    object
 13  Notes        362 non-null    object
 14  utm_source   631 non-null    object
dtypes: object(15)
memory usage: 78.6+ KB


In [200]:
# STEP 1: Merge into canonical "Campaign_ID", "Spend_USD"

df["Campaign_ID"] = df["Campaign_ID"].fillna(df["Campaign Id"])
df["Spend_USD"] = df["Spend_USD"].fillna(df["SpendUSD"])

df = df.drop(columns=["Campaign Id", "SpendUSD"])


In [201]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 670 entries, 0 to 669
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Campaign_ID  501 non-null    object
 1   Date         670 non-null    object
 2   Platform     670 non-null    object
 3   Country      670 non-null    object
 4   Ad_Format    670 non-null    object
 5   Impressions  670 non-null    object
 6   Clicks       641 non-null    object
 7   Spend_USD    670 non-null    object
 8   Conversions  597 non-null    object
 9   CTR          550 non-null    object
 10  Revenue_USD  204 non-null    object
 11  Notes        362 non-null    object
 12  utm_source   631 non-null    object
dtypes: object(13)
memory usage: 68.2+ KB


In [202]:
# STEP 2: Trim whitespace everywhere
for c in df.columns:
    if df[c].dtype == "object":
        df[c] = df[c].str.replace(r"\s+", " ", regex=True).str.strip()

In [203]:
# STEP 3: Standardize categories
def std_platform(x):
    if not isinstance(x, str): return np.nan
    t = x.lower()
    if "google" in t: return "Google Ads"
    if "facebook" in t: return "Facebook"
    if "insta" in t: return "Instagram"
    if "tiktok" in t: return "TikTok"
    if "link" in t: return "LinkedIn"
    return x.title()

def std_country(x):
    if not isinstance(x, str): return np.nan
    t = x.strip().upper()
    mapping = {"USA":"US","UNITED STATES":"US","US":"US",
               "UK":"UK","U K":"UK",
               "AU":"AU","AUS":"AUSTRALIA","DE":"DE","FR":"FR","IN":"IN","PH":"PH"}
    return mapping.get(t, t)

def std_ad_format(x):
    if not isinstance(x, str): return np.nan
    t = x.lower()
    if "video" in t: return "Video"
    if "carousel" in t: return "Carousel"
    if "image" in t or "img" in t: return "Image"
    if "txt" in t or "text" in t: return "Text"
    return x.title()

df["Platform_std"] = df["Platform"].apply(std_platform)
df["Country_std"] = df["Country"].apply(std_country)
df["Ad_Format_std"] = df["Ad_Format"].apply(std_ad_format)


In [204]:
# STEP 4: Parse dates in mixed formats
df["Date_parsed"] = pd.to_datetime(df["Date"], errors="coerce")



In [205]:
# STEP 5: Convert numerics safely
def to_int(x):
    try:
        return int(str(x).replace(",",""))
    except: return np.nan

def to_float(x):
    try:
        return float(str(x).replace(",",""))
    except: return np.nan

def pct_to_float(x):
    if pd.isna(x): return np.nan
    s = str(x).replace("%","")
    try:
        return float(s)/100
    except:
        return np.nan

df["Impressions_num"] = df["Impressions"].apply(to_int)
df["Clicks_num"] = df["Clicks"].apply(to_int)
df["Spend_USD_num"] = df["Spend_USD"].apply(to_float)
df["Conversions_num"] = df["Conversions"].apply(to_int)
df["CTR_num"] = df["CTR"].apply(pct_to_float)


In [206]:
# STEP 6: Fix business logic
# Negative clicks -> NaN
df.loc[df["Clicks_num"] < 0, "Clicks_num"] = np.nan
# Clicks cannot exceed impressions
df.loc[df["Clicks_num"] > df["Impressions_num"], "Clicks_num"] = df["Impressions_num"]
# Conversions cannot exceed clicks
df.loc[df["Conversions_num"] > df["Clicks_num"], "Conversions_num"] = df["Clicks_num"]


In [207]:
# STEP 7: Drop duplicates
key_cols = ["Campaign_ID","Date_parsed","Platform_std","Ad_Format_std","Country_std"]
df = df.drop_duplicates(subset=key_cols, keep="first")


In [208]:
# STEP 8: Final tidy dataset
cleaned = df[[
    "Campaign_ID","Date_parsed","Platform_std","Country_std","Ad_Format_std",
    "Impressions_num","Clicks_num","Spend_USD_num","Conversions_num","CTR_num"
]].rename(columns={
    "Date_parsed":"Date","Platform_std":"Platform","Country_std":"Country",
    "Ad_Format_std":"Ad_Format","Impressions_num":"Impressions",
    "Clicks_num":"Clicks","Spend_USD_num":"Spend_USD","Conversions_num":"Conversions",
    "CTR_num":"CTR"
})

cleaned.to_csv("dirty_marketing_data_cleaned.csv", index=False)
print("Cleaned rows:", len(cleaned))
cleaned.head()


Cleaned rows: 600


,Campaign_ID,Date,Platform,Country,Ad_Format,Impressions,Clicks,Spend_USD,Conversions,CTR
0,CAMP016,2024-06-16,Instagram,CA,Image,3831,99.0,50.07,4.0,0.0482
1,CAMP027,2024-01-11,Instagram,US,Image,8521,209.0,7636.43,12.0,0.0195
2,NaN,NaT,LinkedIn,DE,Video,5795,224.0,908.64,19.0,0.0308
3,CAMP028,2024-07-11,Google Ads,FR,Carousel,7590,7.0,6.59,1.0,NaN
4,NaN,2024-12-01,Google Ads,DE,Carousel,21553,NaN,0.00,0.0,0.0395


In [209]:
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 0 to 668
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Campaign_ID  447 non-null    object        
 1   Date         495 non-null    datetime64[ns]
 2   Platform     600 non-null    object        
 3   Country      600 non-null    object        
 4   Ad_Format    600 non-null    object        
 5   Impressions  600 non-null    int64         
 6   Clicks       563 non-null    float64       
 7   Spend_USD    600 non-null    float64       
 8   Conversions  534 non-null    float64       
 9   CTR          490 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 51.6+ KB
